# Imports

### Import modules

In [1]:
import os
from tqdm import tqdm
import pickle as pkl
import itertools as iter
from matplotlib import pyplot as plt
import sys; sys.path.append('..')
from experiments.functions_Filippo import *
import warnings
import itertools
from IPython.display import Markdown
import io
import pandas as pd
import seaborn as sns
import time


import torch
torch.manual_seed(0);
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader, random_split



### Import local modules

In [2]:
root = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(0, os.path.join(root, 'src'))


from dataset import dataset_dir
from cs import generate_sensing_matrix
from cs.utils import compute_rsnr
from models.unet import UNet

### Hardware settings

In [3]:
gpus = "1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=gpus

os.environ['SCIPY_USE_PROPACK'] = "True"

# limit number of parallel threads numpy spawns
workers_mp = 1 # number of workers for mp.pool
threads_np = "1" # number of threads np generates for every worker

os.environ["OMP_NUM_THREADS"] = threads_np
os.environ["OPENBLAS_NUM_THREADS"] = threads_np
os.environ["MKL_NUM_THREADS"] = threads_np
os.environ["VECLIB_MAXIMUM_THREADS"] = threads_np
os.environ["NUMEXPR_NUM_THREADS"] = threads_np
import numpy as np

In [4]:
def cell_vars(pop=None, verbose=False):
    _bundle = get_cell_vars_as_dict(ipy=get_ipython(), out=io.StringIO(), glob=globals(), offset=0)
    
    if pop is not None and pop in _bundle.keys():
        _bundle.pop(pop)

    for k, v in _bundle.items():
        _b = _bundle.copy()
        _b.pop(k)
        if v==_b or v=={}:
            _bundle.pop(k)
            break;
    
    if verbose: [print(k) for k in _bundle]
    return _bundle

# Multi Unet Exploration

In [5]:
bundle_param = {}

### Dataset parameters

In [8]:
# N = 10_000
N = 2_000_000
# N = 100_000
n = 128
# seed_ecg = 42
seed_ecg = 11
fs = 256 # sampling freq
hr0 = 60 # heart rate low
hr1 = 100 # hear rate high
isnr = 35 # dB (intrinsic snr)

_b = cell_vars(pop='_b')

In [9]:
bundle_param = bundle_param | _b

### A parameters

In [14]:
m = 48
mode_A = ['rakeness']
seed_A = 0
index_A = 0
orth = True
str_corr = '96af96a7ddfcb2f6059092c250e18f2a'
N_try_A = 1_000
loc = 0.25 # localization for rakeness

_b = cell_vars(pop='_b')

In [15]:
bundle_param = bundle_param | _b

### Unet parameters

In [23]:
in_channels = 1
# expanded_channels = [64, 32, 16]
expanded_channels = [32]
# step_number = [4, 3, 2]
step_number = [3]
# kernel_size = [3, 5, 7]
kernel_size = [3]
residual = True
use_batch_norm = False
simple_pool = False
# seed_torch = 0
seed_torch = [0, 1, 2, 3]
# seed_torch = [0, 1, 2, 3]

_b = cell_vars(pop='_b')

In [24]:
bundle_param = bundle_param | _b

### Dataset for A generation

In [25]:
_ecg_N_forA = 10_000
_n_forA = 128
_fs_forA = 256
_hr0_forA = 60
_hr1_forA = 100
_isnr_forA = 35
_seed_forA = 0

_b = cell_vars(pop='_b')

In [26]:
bundle_param_A_dataset = _b.copy()
bundle_param = bundle_param_A_dataset | bundle_param

In [27]:
bundle_param

{'_ecg_N_forA': 10000,
 '_n_forA': 128,
 '_fs_forA': 256,
 '_hr0_forA': 60,
 '_hr1_forA': 100,
 '_isnr_forA': 35,
 '_seed_forA': 0,
 'N': 2000000,
 'n': 128,
 'seed_ecg': 11,
 'fs': 256,
 'hr0': 60,
 'hr1': 100,
 'isnr': 35,
 'm': 48,
 'mode_A': ['rakeness'],
 'seed_A': 0,
 'index_A': 0,
 'orth': True,
 'str_corr': '96af96a7ddfcb2f6059092c250e18f2a',
 'N_try_A': 1000,
 'loc': 0.25,
 'in_channels': 1,
 'expanded_channels': [32],
 'step_number': [3],
 'kernel_size': [3],
 'residual': True,
 'use_batch_norm': False,
 'simple_pool': False,
 'seed_torch': [0, 1, 2, 3]}

#### Select GPU

In [28]:
use_cuda = torch.cuda.is_available()
cuda_index = torch.cuda.device_count() - 1
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(f"Using {device} device")

Using cuda:0 device


### Path and Loading

In [29]:
verbose_load  = False
bundle_all, param_unique_list = unravel_bundle_param(bundle_param)
print('LOADING MODELS:')
model_list = np.array([from_bundle_to_model(b, device, verbose=False) for b in tqdm(bundle_all)])

ind_loaded = model_list!=None

print()
print(f"TOTAL  number of models: {len(model_list)}\n")
print(f"LOADED number of models: {len(model_list[ind_loaded])}\n")

if verbose_load:
    for _m, _p in zip(model_list, param_unique_list):
        if _m is not None:
            display(Markdown(color('LOADED:  ', '', _p, code="00ffff")));
        else:
            display(Markdown(color('MISSING: ', '', _p)));

model_list, param_unique_list = model_list[ind_loaded], np.array(param_unique_list)[ind_loaded]

LOADING MODELS:


100%|██████████| 4/4 [00:01<00:00,  3.18it/s]


TOTAL  number of models: 4

LOADED number of models: 4



### Extract ECG (dataset)

In [30]:
path_ecg = from_bundle_to_ecg_path(bundle_param)

with open(path_ecg, 'rb') as f:
    ecg = pkl.load(f)[:, np.newaxis]


### Extract A (sensing matrix)

In [31]:

dict_A = {}
l_mode_A = bundle_param['mode_A']
if type(l_mode_A) is not list:
    l_mode_A = [l_mode_A]


for _mode_A in l_mode_A:

    _bundle_param = bundle_param.copy()
    _bundle_param['mode_A'] = _mode_A
    path_A = from_bundle_to_A_path(_bundle_param)

    if os.path.isfile(path_A):
        with open(path_A, 'rb') as f:
            A = pkl.load(f)[0]
        print(f"Loading sensing matrix", _mode_A,
                f"(rsnr: {A['rsnr']:.2f}dB (with bpdn) -- seed: {A['seed']})")
        dict_A[_mode_A] = A['matrix']

    else:
        warnings.warn("Sensing matrix generation. "\
                        "Generate with 'best_A_bpdn' for better performances")
        
        path_corr = os.path.join(dataset_dir, 'correlation', corr_str + '.pkl')
        with open(path_corr, 'rb') as f: corr = pkl.load(f)
        
        dict_A[_mode_A] = generate_sensing_matrix(
                    shape = (m, n), 
                    mode=_mode_A, 
                    antipodal=False, 
                    orthogonal=orth,
                    correlation=corr,
                    loc=.25, 
                    seed=seed_A)
        print('generated A', _mode_A)
        print('A shape:', A.shape)

Loading sensing matrix rakeness (rsnr: 27.64dB (with bpdn) -- seed: 30808)


In [32]:
dict_test_loader = {}

for _mode_A, _A in dict_A.items():

    xall = x_to_dataset(ecg, _A, xbar_index=[1, 2])
    
    ds = [(xb_, x_) for xb_, x_ in zip(xall['xbarT'], xall['x'])]

    train_size = int(0.7 * len(ecg))
    val_size = int(0.1 * len(ecg))
    test_size = int(0.2 * len(ecg))
    train_dataset, val_dataset, test_dataset = random_split(ds, [train_size, val_size, test_size])

    # create DataLoaders
    batch_size = 64
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    dict_test_loader[_mode_A] = test_loader

In [33]:
criterion = nn.MSELoss()

In [34]:
list_res = []
for _model, _p_dict in tqdm(zip(model_list, param_unique_list), total=len(model_list)):
    
    _t_loader = dict_test_loader[_p_dict['mode_A']]

    snr_ = 0.0
    test_loss = 0
    with torch.no_grad():
        for inputs, targets in _t_loader:

            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = _model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            
            snr_ += np.mean(compute_rsnr(outputs.detach().cpu().numpy(),
                                        targets.detach().cpu().numpy()))
            
    test_loss /= len(test_loader)
    snr_ /= len(test_loader)

    _p_dict['SNR'] = snr_
    _p_dict['param'] = get_size_model(_model)

    list_res += [_p_dict]

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:51<00:00, 12.83s/it]


In [35]:
snr_max = np.max([i['SNR'] for i in list_res])
param_max = np.max([i['param'] for i in list_res])
for _p_dict in list_res:

    print(f"A: {_p_dict['mode_A']}\t",
            f"ch: {_p_dict['expanded_channels']}\t",
            f"step: {_p_dict['step_number']}\t",
            f"k: {_p_dict['kernel_size']}\t",
            f"seed: {_p_dict['seed_torch']}\t",
            f"SNR: {_p_dict['SNR']:.2f}, ({_p_dict['SNR']/snr_max *100:.1f}%)",
            f"param: {_p_dict['param']:_} ({_p_dict['param']/param_max*100:.1f}%)\t",
            f"index: {(_p_dict['SNR']/snr_max)/(_p_dict['param']/param_max):.2f}")
    
df_res = pd.DataFrame(list_res)

A: rakeness	 ch: 32	 step: 3	 k: 3	 seed: 0	 SNR: 36.17, (100.0%) param: 714_689 (100.0%)	 index: 1.00
A: rakeness	 ch: 32	 step: 3	 k: 3	 seed: 1	 SNR: 36.15, (99.9%) param: 714_689 (100.0%)	 index: 1.00
A: rakeness	 ch: 32	 step: 3	 k: 3	 seed: 2	 SNR: 36.13, (99.9%) param: 714_689 (100.0%)	 index: 1.00
A: rakeness	 ch: 32	 step: 3	 k: 3	 seed: 3	 SNR: 36.15, (99.9%) param: 714_689 (100.0%)	 index: 1.00


In [36]:
df_res

,mode_A,expanded_channels,step_number,kernel_size,seed_torch,SNR,param
0,rakeness,32,3,3,0,36.174161,714689
1,rakeness,32,3,3,1,36.154741,714689
2,rakeness,32,3,3,2,36.134484,714689
3,rakeness,32,3,3,3,36.152376,714689


### Analysis

In [37]:
_df = df_res.groupby(['mode_A', 'expanded_channels', 'step_number', 'kernel_size'])
_df = pd.concat([_df['SNR'].median(), _df['SNR'].std(),_df['param'].median().astype(int),], keys=['SNR median', 'SNR std', 'param'], axis=1)

In [38]:
cm = sns.light_palette("green", as_cmap=True)
_df.style.background_gradient(cmap=cm)

,,,,SNR median,SNR std,param
mode_A,expanded_channels,step_number,kernel_size,,,
rakeness,32,3,3,36.153558,0.016233,714689


In [39]:
_df.unstack('mode_A').style.background_gradient(cmap=cm)

,,,SNR median,SNR std,param
,,mode_A,rakeness,rakeness,rakeness
expanded_channels,step_number,kernel_size,,,
32,3,3,36.153558,0.016233,714689


# Multi Unet Training 

### Log parameters

In [45]:
nohup_type = 'serial' # None or 'serial' or 'parallel'

### Experiment path

In [46]:
# path_py = os.path.join('experiments', 'Unet_xbar.py')
path_py = os.path.join('Unet_xbar.py')

### Parameters

In [68]:
#### DATASET PARAMETERS ####

size_ecg = 2_000_000
# size_ecg = 100_000
n = 128
seed_ecg = 11

#### A PARAMETERS ####

modeA = ['rakeness']
m = 16
str_orth = "True"
seed_A = 0
index_A = 0
str_corr = '96af96a7ddfcb2f6059092c250e18f2a'
N_try_A = 100

#### UNET PARAMETERS ####

in_channels = 1
seed_torch = [0, 1, 2, 3]
expanded_channels = 32
step_number = 3
kernel_size = 3
str_residual = "True"
str_use_batch_norm = "False"
str_simple_pool = "False"
batch_size = 512
lr = 0.001

#### HARDWARE PARAMETERS ####

str_gpus = "1"
threads = 1
workers = 1

#### OTHERS ####
str_retrain = "True"
str_resume_train = "False"

_b = cell_vars(pop='_b')

In [69]:
# dict_bundle_cmd = cell_vars(offset=0)
dict_bundle_cmd = _b.copy()
dict_bundle_cmd

{'size_ecg': 2000000,
 'n': 128,
 'seed_ecg': 11,
 'modeA': ['rakeness'],
 'm': 16,
 'str_orth': 'True',
 'seed_A': 0,
 'index_A': 0,
 'str_corr': '96af96a7ddfcb2f6059092c250e18f2a',
 'N_try_A': 100,
 'in_channels': 1,
 'seed_torch': [0, 1, 2, 3],
 'expanded_channels': 32,
 'step_number': 3,
 'kernel_size': 3,
 'str_residual': 'True',
 'str_use_batch_norm': 'False',
 'str_simple_pool': 'False',
 'batch_size': 512,
 'lr': 0.001,
 'str_gpus': '1',
 'threads': 1,
 'workers': 1,
 'str_retrain': 'True',
 'str_resume_train': 'False'}

### Unravel

In [70]:
dict_bundle_cmd_unravel = unravel_bundle_param(dict_bundle_cmd)[0]
print('number of cmd for bash in the same instruction:', len(dict_bundle_cmd_unravel))

number of cmd for bash in the same instruction: 4


### Crate distinct bash commands

In [71]:
cmd_bash = []
for bundle in dict_bundle_cmd_unravel:
    cmd_bash += [from_bundle_to_cmd_bash(path_py, bundle, nohup=False)]

print("example: first cmd line:")
print(cmd_bash[0])


example: first cmd line:
python Unet_xbar.py --size_ecg 2000000 -n 128 --seed_ecg 11 -m 16 --str_orth True --seed_A 0 --index_A 0 --str_corr 96af96a7ddfcb2f6059092c250e18f2a --N_try_A 100 --in_channels 1 --expanded_channels 32 --step_number 3 --kernel_size 3 --str_residual True --str_use_batch_norm False --str_simple_pool False --batch_size 512 --lr 0.001 --str_gpus 1 --threads 1 --workers 1 --str_retrain True --str_resume_train False --modeA rakeness --seed_torch 0


### File for bash results

In [72]:
now = time.localtime()
now = '_'.join(str(i) for i in [now.tm_year, now.tm_mon, now.tm_mday])
res_single_file = f"nohup_local_gpu={str(str_gpus)}_date={now}.log"
res_global_file = f"nohup_global_gpu={str(str_gpus)}_date={now}.log"
print("res_single_file:", res_single_file)
print("res_global_file:", res_global_file)

res_single_file: nohup_local_gpu=1_date=2024_12_9.log
res_global_file: nohup_global_gpu=1_date=2024_12_9.log


### Merge in one bash command

In [73]:

if nohup_type is not None:
    if nohup_type=='parallel':
        sep_str = f"; nohup "
    elif nohup_type=='serial':
        sep_str = f" >{res_single_file} && "
    else:
        assert False, "nohup_type not in [None, 'parallel' or 'serial'] can be used"
else:
    sep_str = f"; "

cmd_bash_final = sep_str.join(cmd_bash)

if nohup_type is not None:
    if nohup_type=='serial':
        cmd_bash_final = f"nohup sh -c '{cmd_bash_final}'"

if nohup_type:
    cmd_bash_final = cmd_bash_final + f" &> {res_global_file} &"

### Bash commands

In [74]:
print(cmd_bash_final)

nohup sh -c 'python Unet_xbar.py --size_ecg 2000000 -n 128 --seed_ecg 11 -m 16 --str_orth True --seed_A 0 --index_A 0 --str_corr 96af96a7ddfcb2f6059092c250e18f2a --N_try_A 100 --in_channels 1 --expanded_channels 32 --step_number 3 --kernel_size 3 --str_residual True --str_use_batch_norm False --str_simple_pool False --batch_size 512 --lr 0.001 --str_gpus 1 --threads 1 --workers 1 --str_retrain True --str_resume_train False --modeA rakeness --seed_torch 0 >nohup_local_gpu=1_date=2024_12_9.log && python Unet_xbar.py --size_ecg 2000000 -n 128 --seed_ecg 11 -m 16 --str_orth True --seed_A 0 --index_A 0 --str_corr 96af96a7ddfcb2f6059092c250e18f2a --N_try_A 100 --in_channels 1 --expanded_channels 32 --step_number 3 --kernel_size 3 --str_residual True --str_use_batch_norm False --str_simple_pool False --batch_size 512 --lr 0.001 --str_gpus 1 --threads 1 --workers 1 --str_retrain True --str_resume_train False --modeA rakeness --seed_torch 1 >nohup_local_gpu=1_date=2024_12_9.log && python Unet_x

# Unet Parameter exploration

In [ ]:
list_expanded_channels = [64, 32, 16]
list_step_number = [4, 3, 2]
list_kernel_size = [3, 5, 7]

# file_param = 'dummy_Unet_param.pkl'

_unet_many_param(list_expanded_channels, list_step_number, 
                 list_kernel_size, 
                #  file_param,
                 )

{'unet_param': 11521409, 'expanded_channels': 64, 'step_number': 4, 'kernel_size': 3}
{'unet_param': 17796609, 'expanded_channels': 64, 'step_number': 4, 'kernel_size': 5}
{'unet_param': 24071809, 'expanded_channels': 64, 'step_number': 4, 'kernel_size': 7}
{'unet_param': 2860417, 'expanded_channels': 64, 'step_number': 3, 'kernel_size': 3}
{'unet_param': 4417025, 'expanded_channels': 64, 'step_number': 3, 'kernel_size': 5}
{'unet_param': 5973633, 'expanded_channels': 64, 'step_number': 3, 'kernel_size': 7}
{'unet_param': 692609, 'expanded_channels': 64, 'step_number': 2, 'kernel_size': 3}
{'unet_param': 1069569, 'expanded_channels': 64, 'step_number': 2, 'kernel_size': 5}
{'unet_param': 1446529, 'expanded_channels': 64, 'step_number': 2, 'kernel_size': 7}
{'unet_param': 2885313, 'expanded_channels': 32, 'step_number': 4, 'kernel_size': 3}
{'unet_param': 4454145, 'expanded_channels': 32, 'step_number': 4, 'kernel_size': 5}
{'unet_param': 6022977, 'expanded_channels': 32, 'step_number':

### Extract a layer

In [ ]:
unet_param = get_size_model(model)
print(f'total parameter of the Unet: {unet_param}')

total parameter of the Unet: 11521409


In [ ]:
layer_name_list = ['down', 'down_0', 'conv', 'conv_op', '3']
sub_model = get_generic_l(model, layer_name_list)
print(sub_model, '\n\nParam:', get_size_model(sub_model))

Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,)) 

Param: 12352


In [ ]:
layer_name_list = ['down', 'down_0', 'conv']
sub_model = get_generic_l(model, layer_name_list)
print(sub_model, '\n\nParam:', get_size_model(sub_model))

DoubleConv(
  (conv_op): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
) 

Param: 12864


# OTHER

In [42]:
_model = UNet(in_channels=in_channels, num_classes=in_channels,
                expanded_channels=expanded_channels, steps_num=step_number,
                kernel_size=5, residual=residual,
                use_batch_norm=use_batch_norm, simple_pool=simple_pool)

_model.to(device)

UNet(
  (down): DownSeq(
    (down_0): DownSample(
      (conv): DoubleConv(
        (conv_op): Sequential(
          (0): Conv1d(1, 64, kernel_size=(5,), stride=(1,), padding=(2,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
          (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
        )
      )
      (pool): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
    )
    (down_1): DownSample(
      (conv): DoubleConv(
        (conv_op): Sequential(
          (0): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
          (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
          (4): BatchNorm1d(128, eps

In [41]:

_t_loader = dict_test_loader[_p_dict['mode_A']]

snr_ = 0.0
test_loss = 0
with torch.no_grad():
    for inputs, targets in _t_loader:

        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = _model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        
        snr_ += np.mean(compute_rsnr(outputs.detach().cpu().numpy(),
                                    targets.detach().cpu().numpy()))
        
test_loss /= len(test_loader)
snr_ /= len(test_loader)

print(f'{_p_dict} -- SNR: {snr_:.2f}')

{'mode_A': 'rakeness', 'use_batch_norm': False, 'seed_torch': 3} -- SNR: 0.66
